In [ ]:
import time
import requests
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(60)

In [ ]:
def get_all_cookies():
    """获取当前页面所有cookie信息"""
    all_cookies = driver.get_cookies()
    cookie_dict = {}
    for cookie in all_cookies:
        cookie_dict[cookie['name']] = cookie['value']
    return cookie_dict

In [ ]:
get_headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '107',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Origin': 'http://guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

check_headers= {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}


submit_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '126',
    'Content-Type': 'application/json',
    'Cookie': 'Hm_lvt_b21659b538990a950b60a32e93668ae4=1721696256; HMACCOUNT=B11A600C74D21728; edu-s=09c8957c97d0486f9a1c00019fc12c8d; Hm_lpvt_b21659b538990a950b60a32e93668ae4=1721697830',
    'Host': 'guizhou.zxjxjy.com',
    'Origin': 'http://guizhou.zxjxjy.com',
    'Referer': 'http://guizhou.zxjxjy.com/p/classroom/simple?coursewareid=fbb5c5dd51284388abf1ee0f46e91387&courseid=c12ec941a16f47a2aa7d8b869f684cea',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

base_url = "http://guizhou.zxjxjy.com"

In [ ]:
def update_headers(original_headers, new_cookie=None, coursewareid=None, courseid=None):
    # 创建headers的副本，以免修改原始字典
    updated_headers = original_headers.copy()
    
    # 更新cookie（如果提供）
    if new_cookie:
        if isinstance(new_cookie, dict):
            # 如果new_cookie是字典，将其转换为字符串
            cookie_string = '; '.join(f"{k}={v}" for k, v in new_cookie.items())
            updated_headers['Cookie'] = cookie_string
        elif isinstance(new_cookie, str):
            # 如果new_cookie已经是字符串，直接使用
            updated_headers['Cookie'] = new_cookie
        else:
            raise ValueError("new_cookie must be either a dictionary or a string")
    
    # 更新Referer中的coursewareid和courseid（如果提供）
    if coursewareid or courseid:
        referer = updated_headers['Referer']
        # 分割Referer URL
        base_url, params = referer.split('?')
        param_dict = dict(param.split('=') for param in params.split('&'))
        
        # 更新coursewareid和courseid
        if coursewareid:
            param_dict['coursewareid'] = coursewareid
        if courseid:
            param_dict['courseid'] = courseid
        
        # 重建Referer URL
        new_params = '&'.join(f"{k}={v}" for k, v in param_dict.items())
        updated_headers['Referer'] = f"{base_url}?{new_params}"
    
    return updated_headers


In [ ]:

def get_action(courseid, coursewareid, lesson_location,get_headers):
    url = f"{base_url}/p/action/get"
    payload = {
        "courseid": courseid,
        "coursewareid": coursewareid,
        "lessonlocation": lesson_location,
    }
    response = requests.post(url, headers=get_headers, data=payload)
    print(f"GET Action Response: {response.status_code} - {response.text}")

def check_limit(courseid, coursewareid,check_headers):
    url = f"{base_url}/play/checklimit?courseid={courseid}&cwid={coursewareid}"
    response = requests.get(url, headers=check_headers)
    print(f"Check Limit Response: {response.status_code} - {response.text}")

def submit_process(courseid, coursewareid, lesson_location,submit_headers):
    url = f"{base_url}/p/play/submitProcess"
    payload = {
        "courseId": courseid,
        "coursewareId": coursewareid,
        "lessonLocation": lesson_location,
    }
    response = requests.post(url, headers=submit_headers, json=payload)
    print(f"Submit Process Response: {response.status_code} - {response.text}")

def watch_video(video):
    courseid = video["courseid"]
    coursewareid = video["coursewareid"]
    duration = video["duration"]
    
    lesson_location = 0
    last_get_check_time = time.time()
    last_submit_time = time.time()
    start_time = time.time()

    while lesson_location < duration:
        current_time = time.time()
        # 每15秒发送GET和checklimit请求
        if current_time - last_get_check_time >= 15:
            get_action(courseid, coursewareid, lesson_location,update_headers(get_headers))
            check_limit(courseid, coursewareid,update_headers(check_headers))
            check_limit(courseid, coursewareid,update_headers(check_headers))
            last_get_check_time = current_time

        # 每30秒发送submitProcess请求
        if current_time - last_submit_time >= 30:
            lesson_location += 30  # 假设每次播放进度增加30秒
            # 随机增加0.1——1.0秒的播放时间
            lesson_location += round(random.uniform(0.1, 1.0),4)
            submit_process(courseid, coursewareid, lesson_location,update_headers(submit_headers))
            last_submit_time = current_time

        time.sleep(1)  # 每秒钟检查一次


In [ ]:
# 配置
videos = [
    {"courseid": "c12ec941a16f47a2aa7d8b869f684cea", "coursewareid": "889e2848e76844a5ade7dd6c17466b9f", "duration": 2400}, 
    # 添加更多视频
]

In [ ]:
for video in videos:
    watch_video(video)